In [41]:
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import Levenshtein as lev
from concurrent import futures
from IPython.display import clear_output
from tqdm import tqdm
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC

In [9]:
df = pd.read_csv("Senscritique_ids_notes_cleaned.csv")
print(df)

           id                      title                              sc_id  \
0      178014  avatar : la voie de l'eau     avatar_la_voie_de_l_eau/450676   
1      281293   les banshees d'inisherin  les_banshees_d_inisherin/41368328   
2      289305                    tempête                   tempete/44158578   
3      266320                      m3gan                     m3gan/44804275   
4      288544    le tourbillon de la vie   le_tourbillon_de_la_vie/47794530   
...       ...                        ...                                ...   
18339  275748             o fim do mundo            o_fim_do_mundo/39788839   
18340  277796               acceleration              acceleration/41177590   
18341  280139           a perfect family          a_perfect_family/42017435   
18342  281034          jagame thandhiram         jagame_thandhiram/44885972   
18343   51928          délit d'innocence           delit_d_innocence/476236   

       sc_rating                             sc_det

In [48]:
df_sample = df.iloc[0:2000].sample(30, random_state = 0)
df_sample

,id,title,sc_id,sc_rating,sc_detailed_similarity,score_mean
405,292222,the innocents,the_innocents/44876389,6.9,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333
1190,123534,dragons,dragons/448933,7.5,"(1.0, 1.0, 1.0, 1.0)",1.000000
1132,2402,le nom de la rose,le_nom_de_la_rose/451319,7.6,"(1.0, 1.0, 1.0, 1.0)",1.000000
731,12475,american history x,american_history_x/392288,7.8,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333
1754,293472,hit the road,hit_the_road/44882711,7.0,"(1.0, 1.0, 1.0, 1.0)",1.000000
1178,238132,mission impossible - fallout,mission_impossible_fallout/16838981,6.8,"(0.9642857142857143, 1.0, 1.0, 0.9880952380952...",0.988095
1533,29150,la vie de david gale,la_vie_de_david_gale/395550,7.2,"(1.0, 1.0, 1.0, 1.0)",1.000000
1303,196885,12 years a slave,12_years_a_slave/439915,7.3,"(1.0, 0.8, 0.6153846153846154, 0.8051282051282...",0.805128
1857,1665,l'amant,l_amant/388051,6.2,"(1.0, 1.0, 1.0, 1.0)",1.000000
18,294450,les cyclades,les_cyclades/47350870,6.0,"(1.0, 1.0, 1.0, 1.0)",1.000000


In [49]:
def find_on_page(xpath):
    #print('SEARCHING ' + xpath)
    while True:
        try:
            box = driver.find_element(By.XPATH, xpath)
            #box = driver.find_element(By.CSS_SELECTOR, xpath)
            #print('SUCCESS')
            return box
        except:
            pass
        
def get_data_from_sens_critique(driver, sc_id):
    driver.get(f"https:www.senscritique.com/film/{sc_id}/critiques")
    cookies_box = find_on_page('//*[@id="didomi-notice-agree-button"]')
    cookies_box.click()
    review_elements = driver.find_elements(By.LINK_TEXT, "Lire la critique")
    review_ids = [review_element.get_attribute('href').split('/')[-1] for review_element in review_elements]
    i = 0
    for i in range(2):
        try:
            next_page = driver.find_elements(By.CSS_SELECTOR, f"span[data-testid='click-{i + 2}']")[0]
        except:
            # print(f'FAILED AT FINDING {i+2}')
            break
        driver.execute_script("window.scrollTo(0, 400)")
        time.sleep(.3)
        next_page.click()
        # print("passing")
        while all(item in review_elements for item in driver.find_elements(By.LINK_TEXT, "Lire la critique")):
            pass
        # print("found")
        review_elements = driver.find_elements(By.LINK_TEXT, "Lire la critique")
        for element in review_elements:
            if element.get_attribute('href').split('/')[-1] not in review_ids:
                review_ids.append(element.get_attribute('href').split('/')[-1])
    # GET REVIEWS CONTENT
    review_titles = []
    review_authors = []
    review_ratings = []
    review_bodys = []
    for r_id in review_ids:
        response = requests.get(f"https://www.senscritique.com/film/{sc_id.split('/')[0]}/critique/{r_id}")
        review_soup = BeautifulSoup(response.content, 'html.parser')
        review_title = review_soup.find('h1').text
        review_author = review_soup.find('a',
                                         class_='Text__SCText-sc-kgt5u3-0 Username__WrapperUsername-sc-19yskas-1 hrLruW').text
        review_rating = int(review_soup.find('div', class_='Rating__UserRating-sc-1rkvzid-2 fylBAF').text)
        review_body = review_soup.find('div', class_='Content__Container-sc-1a6ct0l-0 kJVhmZ').text
        review_titles.append(review_title)
        review_authors.append(review_author)
        review_ratings.append(review_rating)
        review_bodys.append(review_body)
    return title, rating, list(zip(review_titles, review_authors, review_ratings, review_bodys))

In [55]:
#all_reviews = []
for i in tqdm(range(21, len(df_sample["sc_id"]))):
    print(f"{i}/{len(df_sample['sc_id'])}")
    title = ""
    rating = 0
    reviews = []
    options = webdriver.ChromeOptions()
    #options.add_argument('-headless')
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options = options)
    title, rating, reviews = get_data_from_sens_critique(driver, df_sample["sc_id"].iloc[i])
    all_reviews.append(reviews)

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

21/30


 11%|█████████▎                                                                          | 1/9 [00:27<03:40, 27.59s/it]

22/30


 22%|██████████████████▋                                                                 | 2/9 [01:00<03:36, 30.99s/it]

...

 78%|█████████████████████████████████████████████████████████████████▎                  | 7/9 [03:23<00:58, 29.21s/it]

28/30


 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [03:57<00:30, 30.87s/it]

29/30


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [04:31<00:00, 30.12s/it]


In [56]:
len(all_reviews)

30

In [58]:
df_sample['sc_reviews'] = all_reviews
df_sample

,id,title,sc_id,sc_rating,sc_detailed_similarity,score_mean,sc_reviews
405,292222,the innocents,the_innocents/44876389,6.9,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333,"[(The kids are all bite, Sergent_Pepper, 7, L’..."
1190,123534,dragons,dragons/448933,7.5,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Classique?, StandingFierce, 9, Pas mal de sp..."
1132,2402,le nom de la rose,le_nom_de_la_rose/451319,7.6,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Star Wars 0 : Le Livre qui tue., DjeeVanClee..."
731,12475,american history x,american_history_x/392288,7.8,"(1.0, 0.8, 1.0, 0.9333333333333332)",0.933333,"[(Sabots minables., Sergent_Pepper, 2, Si l’on..."
1754,293472,hit the road,hit_the_road/44882711,7.0,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Jafar Panahi : ""Ceci est un film de mon fils..."
1178,238132,mission impossible - fallout,mission_impossible_fallout/16838981,6.8,"(0.9642857142857143, 1.0, 1.0, 0.9880952380952...",0.988095,"[(Une véritable mission impossible, Behind_the..."
1533,29150,la vie de david gale,la_vie_de_david_gale/395550,7.2,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Parker dévisse, n’opère jamais., Sergent_Pep..."
1303,196885,12 years a slave,12_years_a_slave/439915,7.3,"(1.0, 0.8, 0.6153846153846154, 0.8051282051282...",0.805128,"[(Chaînes payantes..., real_folk_blues, 4, — E..."
1857,1665,l'amant,l_amant/388051,6.2,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Passion sans destin, AudreyAnzu, 6, Cette hi..."
18,294450,les cyclades,les_cyclades/47350870,6.0,"(1.0, 1.0, 1.0, 1.0)",1.000000,"[(Vacances j'oublie tout , Shawn777, 8, Tout c..."


In [59]:
df_sample.to_csv("sc_reviews.csv")